In [1]:
from random import *
import timeit

#Goal was to study the efficiency of different algorithms for matrix multiplication
#The problem with matrix multiplication is that naive algorithm is not very efficient for big matrices
#We must use other algorithm to have correct execution time
#Julien : Cache Behavior
#Gautier : Strassen Algorithm
#Arnaud : Naive & Naive+ & Data Generation & Debug Strassen + Cache Behavior

def set_size_of_matrix(n,m,p):
    A = [[0 for x in range(m)] for y in range(n)]
    B = [[0 for x in range(p)] for y in range(m)]
    C = [[0 for x in range(p)] for y in range(n)]
    return A,B,C

#Generation of data for matrices
def random_fill_matrix(M):
    for i in range(len(M)):
        for j in range(len(M[0])):
            M[i][j] = randint(0, 100)
    return M

#O(n^3)
#Naive algorithm
def algo_iterative(A,B,C):
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                C[i][j] += A[i][k] * B[k][j]
    return C

#O(n^3)
#Naive algorithm + 
#Switch between two loop make the cache access quicker
def algo_iterative_plus(A,B,C):
    for i in range(len(A)):
        for k in range(len(B)):
            for j in range(len(B[0])):
                C[i][j] += A[i][k] * B[k][j]
    return C

#Split 1 matrix into 4 smaller matrices
def split_of_matrix(M):
    half = len(M)//2
    M11 = [[M[x][y] for y in range(0,half)] for x in range(0,half)]
    M12 = [[M[x][y] for y in range(half,len(M))] for x in range(0,half)]
    M21 = [[M[x][y] for y in range(0,half)] for x in range(half,len(M))]
    M22 = [[M[x][y] for y in range(half,len(M))] for x in range(half,len(M))]
    return M11,M12,M21,M22

#Addition of matrices
def add_of_matrix(A,B):
    return [[A[x][y]+B[x][y] for y in range(len(A))] for x in range(len(A))]

#Substraction of matrices
def sub_of_matrix(A,B):
    return [[A[x][y]-B[x][y] for y in range(len(A))] for x in range(len(A))]

#O(n^2.8074)
#Only work with square matrix
def strassen(A,B):
    if len(A) == 1:
        return [[A[0][0]*B[0][0]]]
    #LEAF_SIZE when the split matrix size is below
    #the algorithm will use naive algorithm which is faster for small matrix multiplication
    elif len(A) <= 64:
        C = [[0 for x in range(0,len(A))] for y in range(0,len(A))]
        for i in range(len(A)):
          for j in range(len(B[0])):
              for k in range(len(B)):
                  C[i][j] += A[i][k] * B[k][j]
        return C
    else:
        #Split the matrices in four pieces
        A11,A12,A21,A22 = split_of_matrix(A)
        B11,B12,B21,B22 = split_of_matrix(B)
        
        #Definition of new matrices
        M1 = strassen(add_of_matrix(A11,A22),add_of_matrix(B11,B22))
        M2 = strassen(add_of_matrix(A21,A22),B11)
        M3 = strassen(A11,sub_of_matrix(B12,B22))
        M4 = strassen(A22,sub_of_matrix(B21,B11))
        M5 = strassen(add_of_matrix(A11,A12),B22)
        M6 = strassen(sub_of_matrix(A21,A11),add_of_matrix(B11,B12))
        M7 = strassen(sub_of_matrix(A12,A22),add_of_matrix(B21,B22))
        
        #Recombine the matrices to make submatrix of C
        C11 = add_of_matrix(sub_of_matrix(add_of_matrix(M1,M4),M5),M7)
        C12 = add_of_matrix(M3,M5)
        C21 = add_of_matrix(M2,M4)
        C22 = add_of_matrix(add_of_matrix(sub_of_matrix(M1,M2),M3),M6)

        C = [[0 for x in range(0,len(A))] for y in range(0,len(A))]
        
        #Recombine the C matrix
        for i in range(len(C11)):
            for j in range(len(C11)):
                C[i][j] = C11[i][j]
                C[i][j+len(C11)] = C12[i][j]
                C[i+len(C11)][j] = C21[i][j]
                C[i+len(C11)][j+len(C11)] = C22[i][j]

    return C

#O(n^3)
#This algorithm is supposed to prevent the domination of cache misses in the 
#computationnal time. tileSize must be set as an O(sqrt(M)) where M
#is the Byte size of the CPU cache
def algo_CacheBehavior(A,B,C,tileSize):
    if len(A) == 1:
        return [[A[0][0]*B[0][0]]]
    else:
      #This part is the first classic iterative run
      for i in range(1,len(A),tileSize):
          for j in range(1,len(B[0]),tileSize):
              for k in range(1,len(B),tileSize):

                  #This part is where we do a localized run to minimize
                  #The misses in caches
                  for a in range(i-1,min(i-1+tileSize,len(A))):
                      for b in range(j-1,min(j-1+tileSize,len(B[0]))):
                          for c in range(k-1,min(k-1+tileSize,len(B))):
                              C[a][b] += A[a][c] * B[c][b]
    return C

#Check if every values of matrices are egals
def test_of_result(A,B):
    for i in range(len(A)):
      for j in range(len(A[0])):
        assert(A[i][j]==B[i][j]),"Matrix are not equal, they should be"
    return "Matrix are equal as they should"
    

if __name__ == "__main__":

    #Strassen can only use square matrix and multiple of 2
    n = 512
    A,B,C = set_size_of_matrix(n,n,n)
    D,E,F = set_size_of_matrix(n,n,n)
    A = random_fill_matrix(A)
    B = random_fill_matrix(B)

    #print('A =')
    #for r in A:
    #    print(r)
    #print('B =')
    #for r in B:
    #    print(r)    

    start = timeit.default_timer()
    C = algo_iterative(A,B,C)
    stop = timeit.default_timer()
    print('Iterative Time: ', stop - start)
    #for r in C:
    #    print(r)
    start = timeit.default_timer()
    F = algo_iterative_plus(A,B,F)
    stop = timeit.default_timer()
    print('Iterative+ Time: ', stop - start)
    #for r in C:
    #    print(r)
    #We must enter an actual sound cache value
    start = timeit.default_timer()
    D = algo_CacheBehavior(A,B,D,60)
    stop = timeit.default_timer()
    print('Cache Time: ', stop - start)
    #for r in D:
    #    print(r)
    start = timeit.default_timer()
    E = strassen(A,B)
    stop = timeit.default_timer()
    print('Strassen Time: ', stop - start)
    #for r in E:
    #    print(r)
    print(test_of_result(C,D))
    print(test_of_result(C,F))
    print(test_of_result(C,E))
    print(test_of_result(D,E))

#Naive algorithm is good for small matrix multiplication
#The Naive+ algorithm is better when we start to multiply bigger matrices
#Cache Behavior Algorithm is even better for big matrices
#However Strassen is the best we implement for big matrices, but very bad for small matrices
#So in conclusion we should use the implementation oof Strassen
#Because when the matrices are small, it do not use Strassen and instead it use the naive algorithm

Iterative Time:  80.80978619999951
Iterative+ Time:  73.73034039999766
Cache Time:  75.75566499999695
Strassen Time:  53.39396690000285
Matrix are equal as they should
Matrix are equal as they should
Matrix are equal as they should
Matrix are equal as they should
